In [ ]:
from osg.utils.polyform_utils import *
from PIL import Image
import json
import numpy as np
import os
import tqdm
import matplotlib.pyplot as plt
import cv2
import numpy as np
import open3d as o3d
from scipy.spatial.transform import Rotation as R

In [ ]:
class LimpConvertor():
    """Converts Polycam data to the format expected by Instant-NGP"""
    def __init__(self, corrected_image_padding: int = 5):
        self.corrected_image_padding = corrected_image_padding
        self.observation_data = {'images':{},
        'poses':{},
        'depth_data':{},
        'rep_pose':{}}

    def convert(self, folder: CaptureFolder, output_path: str = "limp_dataset"):
        """
        Converts a Polycam CaptureFolder into a format LIMP can consume [images, depths and poses]

        Args:
            folder: the capture folder to convert
            output_path: the path to write the transforms.json file. By default this is
                written at the root of the CaptureFolder's data directory.
        """
        data = {}
        keyframes = folder.get_keyframes(rotate=True)
        if len(keyframes) == 0:
            logger.warning("Capture folder does not have any data! Aborting conversion to Instant NGP")
            return

        bbox = CaptureFolder.camera_bbox(keyframes)
        print(bbox)
        ## Use camera bbox to compute scale, and offset 
        ## See https://github.com/NVlabs/instant-ngp/blob/master/docs/nerf_dataset_tips.md#scaling-existing-datasets
        max_size = np.max(bbox.size()) * 0.75
        data["scale"] = float(1.0 / max_size)
        offset = -bbox.center() / max_size
        data["offset"] = [float(offset[0]) + 0.5, float(offset[1]) + 0.5, float(offset[2]) + 0.5]
        data["aabb_scale"] = 2

        ## add the frames
        frames = []
        for keyframe in tqdm.tqdm(keyframes):
            frames.append(self._convert_keyframe(keyframe, folder))

        data["frames"] = frames
        print("done")
        return data


    def _convert_keyframe(self, keyframe: Keyframe, folder: CaptureFolder) -> dict:
        """ Converts Polycam keyframe into a dictionary to be serialized as json """
        frame = {}
        frame["id"] = keyframe.timestamp

        #add frame camera intrinsics
        cam = keyframe.camera
        frame["fl_x"] = cam.fx
        frame["fl_y"] = cam.fy
        if folder.has_optimized_poses():
            # For optimized data we apply a crop of the image data, and we need to update the intrinsics here to match
            frame["cx"] = cam.cx - self.corrected_image_padding
            frame["cy"] = cam.cy - self.corrected_image_padding
            frame["w"] = cam.width - 2 * self.corrected_image_padding
            frame["h"] = cam.height - 2 * self.corrected_image_padding
        else:
            frame["cx"] = cam.cx 
            frame["cy"] = cam.cy
            frame["w"] = cam.width
            frame["h"] = cam.height

        #add depth data
        frame["depth_map_path"] = "./{}/{}.png".format(CaptureArtifact.DEPTH_MAPS.value, keyframe.timestamp)
        
        # iphone Polycam depth data is stored in lossless .png format as a single channel image where depth is encoded as a 16-bit integer with units of millimeters, so a value of 1250 would correspond to 1.25 meters.
        depth_path = os.path.join(folder.root, CaptureArtifact.DEPTH_MAPS.value, "{}.png".format(keyframe.timestamp))
        depth_img = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)
        frame["depth_image"] = depth_img

        #add frame pose data
        frame_camera_pose = keyframe.camera.get_pose()
        frame["position_x"] = frame_camera_pose['position_x']
        frame["position_y"] = frame_camera_pose['position_y']
        frame["position_z"] = frame_camera_pose['position_z']
        frame["quaternion_x"] = frame_camera_pose['quaternion_x']
        frame["quaternion_y"] = frame_camera_pose['quaternion_y']
        frame["quaternion_z"] = frame_camera_pose['quaternion_z']
        frame["quaternion_w"] = frame_camera_pose['quaternion_w']

        # add frame image
        if folder.has_optimized_poses():
            # For the corrected images we apply an image crop to remove the black strip around the boundary that is a result of the undistortion process
            img_full_path = os.path.join(folder.root, CaptureArtifact.CORRECTED_IMAGES.value, "{}.jpg".format(keyframe.timestamp))
            img_full_path_crop = os.path.join(folder.root, CaptureArtifact.CORRECTED_IMAGES.value, "{}_crop.jpg".format(keyframe.timestamp))
            im = Image.open(img_full_path)
            im = im.crop((self.corrected_image_padding, self.corrected_image_padding, keyframe.camera.width - 2 * self.corrected_image_padding, keyframe.camera.height - 2 * self.corrected_image_padding))
            im.save(img_full_path_crop)
            frame["image_file_path"] = "./{}/{}_crop.jpg".format(CaptureArtifact.CORRECTED_IMAGES.value, keyframe.timestamp)
            frame["image"] = np.asarray(im)
        else:
            frame["file_path"] = "./{}/{}.jpg".format(CaptureArtifact.IMAGES.value, keyframe.timestamp)
            frame["image"] = np.asarray(Image.open(os.path.join(folder.root, CaptureArtifact.IMAGES.value, "{}.jpg".format(keyframe.timestamp))))
        if keyframe.camera.blur_score:
            frame["sharpness"] = keyframe.camera.blur_score
        frame["transform_matrix"] = keyframe.camera.transform_rows
        return frame

In [ ]:
data_folder_path = "../data/polyform_sample_data2"
folder = CaptureFolder(data_folder_path)
convertor = LimpConvertor()
transformed_data = convertor.convert(folder)

In [ ]:
#view sample image
data_frame = transformed_data['frames'][0]
data_frame

In [ ]:
#plot image and dept map side by side
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(data_frame['image'])
plt.title("Image")
plt.subplot(1, 2, 2)
plt.imshow(data_frame['depth_image'])
plt.title("Depth Map")
plt.show()

In [ ]:
#exposed to compute rotation matrix from quaternion
def rotation_matrix_from_quaternion_manual(quaternion):
   #quaternion format: (qw, qx, qy, qz)

    # Step 1: Normalize the quaternion
   quaternion = quaternion / np.linalg.norm(quaternion)
    
   # Step 2: Extract quaternion components
   w, x, y, z = quaternion                              

   # Step 3: Construct rotation matrix
   R = np.array([[1 - 2 * y ** 2 - 2 * z ** 2, 2 * x * y - 2 * w * z, 2 * x * z + 2 * w * y],
               [2 * x * y + 2 * w * z, 1 - 2 * x ** 2 - 2 * z ** 2, 2 * y * z - 2 * w * x],
               [2 * x * z - 2 * w * y, 2 * y * z + 2 * w * x, 1 - 2 * x ** 2 - 2 * y ** 2]])
   return R


def rotation_matrix_from_quaternion(quaternion):
    #quartanion format: (qx, qy, qz, qw)
    r = R.from_quat(quaternion)
    return r.as_matrix()

def read_images_and_params(transformed_data, frame=0):
    depth_img = transformed_data['frames'][frame]['depth_image']
    depth_img = np.nan_to_num(depth_img, nan=0.0, posinf=0.0, neginf=0.0)
    color_img = transformed_data['frames'][frame]['image']
    
    CX = transformed_data['frames'][frame]['cx']
    CY = transformed_data['frames'][frame]['cy']
    FX = transformed_data['frames'][frame]['fl_x']
    FY = transformed_data['frames'][frame]['fl_y']
    
    # quart = [transformed_data['frames'][frame]['quaternion_x'],
    #          transformed_data['frames'][frame]['quaternion_y'],
    #          transformed_data['frames'][frame]['quaternion_z'],
    #          transformed_data['frames'][frame]['quaternion_w']]

    quart = [transformed_data['frames'][frame]['quaternion_w'],
             transformed_data['frames'][frame]['quaternion_x'],
             transformed_data['frames'][frame]['quaternion_y'],
             transformed_data['frames'][frame]['quaternion_z']]
    
    datapoint_rotation_matrix = rotation_matrix_from_quaternion(quart)
    datapoint_position = np.array([transformed_data['frames'][frame]['position_x'],
                                   transformed_data['frames'][frame]['position_y'],
                                   transformed_data['frames'][frame]['position_z']])
    
    # affine_matrix = np.array(
    #     [
    #         [1, 0, 0, 0],
    #         [0, -1, 0, 0],
    #         [0, 0, -1, 0],
    #         [0, 0, 0, 1],
    #     ]
    # )
    # datapoint_position = datapoint_position @ affine_matrix

    # # Converts poses from (X, Z, Y) to (X, -Y, Z).
    # affine_matrix = np.array(
    #     [
    #         [1, 0, 0, 0],
    #         [0, 0, -1, 0],
    #         [0, 1, 0, 0],
    #         [0, 0, 0, 1],
    #     ]
    # )
    # datapoint_position = affine_matrix @ datapoint_position
    
    return depth_img, color_img, CX, CY, FX, FY, datapoint_rotation_matrix, datapoint_position

def resize_color_image_and_scale_intrinsics(color_img, depth_img, CX, CY, FX, FY):
    H, W = depth_img.shape
    H_orig, W_orig = color_img.shape[:2]
    
    scale_x = W / W_orig
    scale_y = H / H_orig
    
    CX_new = CX * scale_x
    CY_new = CY * scale_y
    FX_new = FX * scale_x
    FY_new = FY * scale_y
    
    if H != H_orig or W != W_orig:
        color_img = cv2.resize(color_img, (W, H), interpolation=cv2.INTER_AREA)
    
    return color_img, CX_new, CY_new, FX_new, FY_new

def backproject_worldframe(i, j, depth, cx, cy, fx, fy, rotation_matrix, position):
    z = depth / 1000.0  # Convert millimeters to meters
    x = (j - cx) * z / fx
    y = (i - cy) * z / fy
    point_camera_frame = np.array([x, y, z])
    point_world_frame = rotation_matrix @ point_camera_frame
    return point_world_frame, point_camera_frame

def generate_point_cloud(depth_img, color_img, CX, CY, FX, FY, rotation_matrix, position):
    H, W = depth_img.shape
    total_pcds = []
    total_colors = []
    for i in range(H):
        for j in range(W):
            if depth_img[i, j] > 0:  # Only consider points with valid depth
                transformed_xyz, _ = backproject_worldframe(i, j, depth_img[i, j], CX, CY, FX, FY, rotation_matrix, position)
                total_pcds.append(transformed_xyz)
                total_colors.append(color_img[i, j] / 255.0)
    return total_pcds, total_colors

def fuse_point_clouds(all_pcds, all_colors):
    fused_pcd = o3d.geometry.PointCloud()
    fused_pcd.points = o3d.utility.Vector3dVector(all_pcds)
    fused_pcd.colors = o3d.utility.Vector3dVector(all_colors)
    return fused_pcd

def fuse_point_clouds_with_voxel_filter(all_pcds, all_colors, voxel_size=0.01):
    # Create an empty point cloud
    fused_pcd = o3d.geometry.PointCloud()
    fused_pcd.points = o3d.utility.Vector3dVector(all_pcds)
    fused_pcd.colors = o3d.utility.Vector3dVector(all_colors)
    
    # Apply voxel grid filter to downsample and merge points
    fused_pcd = fused_pcd.voxel_down_sample(voxel_size=voxel_size)
    
    return fused_pcd

def save_point_cloud(point_cloud, filename="fused_pointcloud.ply"):
    o3d.io.write_point_cloud(filename, point_cloud)
    print(f"Point cloud saved to {filename}")

def save_single_point_cloud(total_pcds, total_colors, filename="single_pointcloud.ply"):
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(total_pcds)
    point_cloud.colors = o3d.utility.Vector3dVector(total_colors)
    o3d.io.write_point_cloud(filename, point_cloud)
    print(f"Point cloud saved to {filename}")

## generate and save one pointcloud
# frame = 0
# depth_img, color_img, CX, CY, FX, FY, rotation_matrix, position = read_images_and_params(transformed_data, frame=frame)
# color_img, CX, CY, FX, FY = resize_color_image_and_scale_intrinsics(color_img, depth_img, CX, CY, FX, FY)
# pcds, colors = generate_point_cloud(depth_img, color_img, CX, CY, FX, FY, rotation_matrix, position)
# save_single_point_cloud(pcds,colors)

## generate and save fused pointcloud
## Main execution
all_pcds = []
all_colors = []

for frame in tqdm.tqdm((range(len(transformed_data['frames'])))):
    depth_img, color_img, CX, CY, FX, FY, rotation_matrix, position = read_images_and_params(transformed_data, frame=frame)
    color_img, CX, CY, FX, FY = resize_color_image_and_scale_intrinsics(color_img, depth_img, CX, CY, FX, FY)
    pcds, colors = generate_point_cloud(depth_img, color_img, CX, CY, FX, FY, rotation_matrix, position)
    ## Visualize individual frame point cloud (optional for debugging)
    # visualize_point_cloud(pcds, colors, title=f"Frame {frame_index} Point Cloud")
    all_pcds.extend(pcds)
    all_colors.extend(colors)

# fused_pcd = fuse_point_clouds(all_pcds, all_colors)
fused_pcd = fuse_point_clouds_with_voxel_filter(all_pcds, all_colors, voxel_size=0.01)
save_point_cloud(fused_pcd)
